In [ ]:
import matplotlib.pyplot as mplt
import pandas as pd
import numpy as np
import geopandas as gpd
from functools import partial
#from tqdm import tqdm

from helper import DataHandler, Plot, plot_grid
from functools import partial

from partition import Partition
from markovchain import SingleMetricOptimizer

from markovchain import recom

from falcomchain.grid import Grid


In [ ]:
from graph import Graph
import networkx as nx
from graph import Graph

def tree_with_twenty_nodes():
    """Returns a tree that looks like this:
    
    1 - 2 - 3 - 4
        |       |
    5 - 6 - 7   8 - 9 - 10
    |       |       |
    11-12   13      14
    |
    15-16-17
    |
    18-19
    |
    20
    
    """
    graph = Graph()
    graph.add_edges_from(
        [
            (1, 2),(2, 3),(3, 4),
            (2, 6),(4, 8),
            (5, 6),(6, 7),(8, 9),(9, 10),
            (5, 11),(7, 13),(9, 14),
            (11, 12),
            (11, 15),
            (15, 16),(16, 17),
            (15, 18),
            (18, 19),
            (18, 20),
        ]
    )
    return graph

def find_successors(graph: Graph, root):
        return {a: b for a, b in nx.bfs_successors(graph, root)}

h = tree_with_twenty_nodes()
succ = find_successors(graph=h, root=2)
print(succ)

ModuleNotFoundError: No module named 'graph'

In [ ]:
grid_half = Grid(dimensions=(20,20), num_candidates=50, density='opposite', threshold=(10,0))
graph_half = grid_half.graph
plot_grid(grid_half.graph)

In [ ]:
grid_uniform = Grid(dimensions=(20,20), num_candidates=50, density='uniform', threshold=(10,0))
graph_uniform = grid_uniform.graph
plot_grid(grid_uniform.graph)

In [ ]:
grid_opposite = Grid(dimensions=(20,20), num_candidates=50, density='opposite', threshold=(10,10))
graph_opposite = grid_opposite.graph
plot_grid(grid_opposite.graph)

In [ ]:
# if k_1 <= x <= m - k_1 or k_2 <= y <= n - k_2, then pop is 30. Here, theresold = (k_1, k_2) and dimensions=(m,n)
grid_corners = Grid(dimensions=(20,20), num_candidates=50, density='corners', threshold=(5, 5))
graph_corners = grid_corners.graph
plot_grid(grid_corners.graph)

In [6]:
import travel_time as travel

def travel_times(graph):
    candidates = {node for node in graph.nodes if graph.nodes[node]['candidate']==True}
    for u, v in graph.edges():
        graph[u][v]['time'] = 10
    travels = travel.travel_time(graph, candidates)
    return travels

travels = travel_times(graph_half)

In [5]:
"Initial partition"
pop = sum(graph.nodes[node]['population'] for node in graph.nodes)
area = sum(graph.nodes[node]['area'] for node in graph.nodes)   
#area 
#avg_density = area / pop

# nodelarin density check i burda yapilip sonuc attribute olarak atansin. Attribute ismi simdilik density

column_names = ['population', 'area', 'candidate', 'avg_density'] # avg_density attribute ata. If density is not considered, then we have to pass false.
#check_density = ?

import partition
my_updaters = {
    "population": partition.Tally(column_names[0], alias="population"),
    "cut_edges": partition.cut_edges,
    "perimeter": partition.perimeter,
    "area": partition.Tally("area", alias="area"),}

In [ ]:
initial_solution = Partition.from_random_assignment(
travel_times=travels,
graph=graph,
capacity_level = 1,
epsilon=0.01,
column_names = column_names,
pop_target = pop // 4,
#density = 0.5,
updaters=my_updaters)